In [1]:
import numpy as np

In [2]:
Cij_example = np.array(
    [
        [96.2, 46.1, 38.4, 5.9, -0.2, -0.4],
        [46.1, 189.4, 15.4, -7.0, -5.1, -6.8],
        [38.4, 15.4, 171.9, 2.2, 7.2, -9.8],
        [5.9, -7.0, 2.2, 23.6, -1.1, -4.8],
        [-0.2, -5.1, 7.2, -1.1, 33.1, 1.4],
        [-0.4, -6.8, -9.8, -4.8, 1.4, 35.5],
    ]
)


In [3]:
def rotate_Cij(
    stiffness_matrix: np.ndarray,
    angle_degrees: float,
    rotation_axis: int = 3
    ) -> tuple[np.ndarray, np.ndarray]:
    """
    Rotates the 6x6 stiffness matrix in Voigt notation using the Bond transformation matrix.

    Parameters
    ----------
    stiffness_matrix : np.ndarray
        The original 6x6 stiffness matrix.

    angle_degrees : float
        The rotation angle in degrees

    rotation_axis : int, optional
        The axis around which the rotation is performed:
        - 1: Rotate around the 1-axis (fix 1-axis, rotate 2 and 3 axes).
        - 2: Rotate around the 2-axis (fix 2-axis, rotate 1 and 3 axes).
        - 3: Rotate around the 3-axis (fix 3-axis, rotate 1 and 2 axes).
        Default is 3.

    Returns
    -------
    np.ndarray
        The rotated 6x6 stiffness matrix.

    References
    ----------
    - Bond, W., Jan. 1943, The mathematics of the physical properties of crystals,
      The Bell System Technical Journal, 1-72.
    """

    # Convert angle from degrees to radians
    theta = np.deg2rad(angle_degrees)

    # Calculate cosines and sines
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    if rotation_axis == 1:
        # Rotation around the 1-axis (x1 is fixed)
        bond_matrix = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, cos_theta**2, sin_theta**2, 0, 0, 2*cos_theta*sin_theta],
            [0, sin_theta**2, cos_theta**2, 0, 0, -2*cos_theta*sin_theta],
            [0, 0, 0, cos_theta, -sin_theta, 0],
            [0, 0, 0, sin_theta, cos_theta, 0],
            [0, -cos_theta*sin_theta, cos_theta*sin_theta, 0, 0, cos_theta**2 - sin_theta**2]
        ])
    elif rotation_axis == 2:
        # Rotation around the 2-axis (x2 is fixed)
        bond_matrix = np.array([
            [cos_theta**2, 0, sin_theta**2, 0, 2*cos_theta*sin_theta, 0],
            [0, 1, 0, 0, 0, 0],
            [sin_theta**2, 0, cos_theta**2, 0, -2*cos_theta*sin_theta, 0],
            [0, 0, 0, cos_theta, 0, sin_theta],
            [-cos_theta*sin_theta, 0, cos_theta*sin_theta, 0, cos_theta**2 - sin_theta**2, 0],
            [0, 0, 0, -sin_theta, 0, cos_theta]
        ])
    elif rotation_axis == 3:
        # Rotation around the 3-axis (x3 is fixed)
        bond_matrix = np.array([
            [cos_theta**2, sin_theta**2, 0, 0, 0, 2*cos_theta*sin_theta],
            [sin_theta**2, cos_theta**2, 0, 0, 0, -2*cos_theta*sin_theta],
            [0, 0, 1, 0, 0, 0],
            [0, 0, 0, cos_theta, -sin_theta, 0],
            [0, 0, 0, sin_theta, cos_theta, 0],
            [-cos_theta*sin_theta, cos_theta*sin_theta, 0, 0, 0, cos_theta**2 - sin_theta**2]
        ])
    else:
        raise ValueError("rotation_axis must be either 1, 2, or 3")

    # Apply the Bond transformation to the stiffness matrix
    rotated_stiffness_matrix = bond_matrix @ stiffness_matrix @ bond_matrix.T

    return rotated_stiffness_matrix, bond_matrix


def rotate_Cij2(
        stiffness_matrix: np.ndarray,
        angle_degrees: float,
        rotation_axis: int = 3
    ) -> tuple[np.ndarray, np.ndarray]:
    """
    Rotate a 6x6 stiffness matrix in Voigt notation using the Bond transformation method.

    This function applies a coordinate transformation to the input stiffness matrix
    by rotating it around a specified axis. It uses the Bond transformation matrix
    to perform the rotation.

    Parameters:
    -----------
    stiffness_matrix : np.ndarray
        The original 6x6 stiffness matrix in Voigt notation.
    angle_degrees : float
        The rotation angle in degrees (positive for counterclockwise rotation).
    rotation_axis : int, optional (default=3)
        The axis around which to rotate:
        1: Rotate around x1-axis (fix x1, rotate x2 and x3)
        2: Rotate around x2-axis (fix x2, rotate x1 and x3)
        3: Rotate around x3-axis (fix x3, rotate x1 and x2)

    Returns:
    --------
    tuple[np.ndarray, np.ndarray]
        - The rotated 6x6 stiffness matrix
        - The Bond transformation matrix used for rotation

    Raises:
    -------
    ValueError
        If the input matrix is not 6x6 or if an invalid rotation axis is specified.

    Notes:
    ------
    - The function uses the Bond transformation method, which is particularly useful
      for rotating elastic constant tensors in crystallography and rock physics.
    - The rotation is performed in the right-handed coordinate system.
    - For anisotropic media, different rotation axes can be used to analyze
      various orientations of symmetry planes.

    References:
    -----------
    Bond, W. (1943). The mathematics of the physical properties of crystals. 
    The Bell System Technical Journal, 22(1), 1-72.
    """
    # Input validation
    if stiffness_matrix.shape != (6, 6):
        raise ValueError("Input stiffness matrix must be 6x6")
    if rotation_axis not in [1, 2, 3]:
        raise ValueError("Rotation axis must be 1, 2, or 3")

    # Convert angle to radians and calculate trigonometric functions
    theta = np.deg2rad(angle_degrees)
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    cos2_theta, sin2_theta = cos_theta**2, sin_theta**2

    # Define common terms for Bond matrices
    common_terms = np.array([
        [cos2_theta, sin2_theta, 0, 0, 0, 2*cos_theta*sin_theta],
        [sin2_theta, cos2_theta, 0, 0, 0, -2*cos_theta*sin_theta],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, cos_theta, -sin_theta, 0],
        [0, 0, 0, sin_theta, cos_theta, 0],
        [-cos_theta*sin_theta, cos_theta*sin_theta, 0, 0, 0, cos2_theta - sin2_theta]
    ])

    # Create Bond matrix based on rotation axis
    if rotation_axis == 1:
        bond_matrix = np.roll(common_terms, 2, axis=0)
        bond_matrix = np.roll(bond_matrix, 2, axis=1)
    elif rotation_axis == 2:
        bond_matrix = np.roll(common_terms, 1, axis=0)
        bond_matrix = np.roll(bond_matrix, 1, axis=1)
    else:  # rotation_axis == 3
        bond_matrix = common_terms

    # Apply the Bond transformation to the stiffness matrix
    rotated_stiffness_matrix = bond_matrix @ stiffness_matrix @ bond_matrix.T

    return rotated_stiffness_matrix, bond_matrix


def rotate_Cij_fromMatlab(c: np.ndarray, theta: float) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculate the Bond transformation of the elastic stiffness matrix.

    This function rotates the stiffness tensor around the vertical axis (x3).
    The old axes are x1 (horizontal), x2 (horizontal), x3 (vertical).
    The new axes are x1' (horizontal), x2' (horizontal), x3' = x3 (vertical).
    The vertical axis stays the same, while the x1 axis rotates counter-clockwise.

    Parameters:
    -----------
    c : np.ndarray
        The 6x6 stiffness tensor (cij, i,j=1 to 6) under the old coordinate system.
    theta : float
        The angle (in degrees) required to rotate x1 to x1'.

    Returns:
    --------
    tuple[np.ndarray, np.ndarray]
        cnew : The stiffness tensor under the new coordinate system.
        M : The Bond transformation matrix.

    Notes:
    ------
    - This transformation may be applied to TIH (Transverse Isotropic with 
      Horizontal symmetry axis) media, e.g., that caused by vertically aligned fractures.
    - The angle theta may be assigned to be the angle between the fracture normal 
      and a seismic line.
    - To test validity, apply this function twice to an arbitrary 6x6 matrix,
      first by degree phi, then by degree -phi, to see if the original matrix is recovered.

    References:
    -----------
    - Rock Physics Handbook, Chapter 1.4, "Coordinate Transformations"
    - Originally written by Haibin Xu, 2002
    - Modified by G. Mavko, 2003
    """
    if c.shape != (6, 6):
        raise ValueError("Input stiffness matrix must be 6x6")

    # Convert angle to radians
    theta = np.deg2rad(theta)

    # Calculate the cosines
    b11 = b22 = np.cos(theta)
    b33 = 1
    b21 = np.cos(theta + np.pi/2)
    b12 = np.cos(np.pi/2 - theta)
    b13 = b31 = b23 = b32 = 0

    # Calculate the Bond matrix
    M1 = np.array([[b11**2, b12**2, b13**2],
                   [b21**2, b22**2, b23**2],
                   [b31**2, b32**2, b33**2]])

    M2 = np.array([[b12*b13, b13*b11, b11*b12],
                   [b22*b23, b23*b21, b21*b22],
                   [b32*b33, b33*b31, b31*b32]])

    M3 = np.array([[b21*b31, b22*b32, b23*b33],
                   [b31*b11, b32*b12, b33*b13],
                   [b11*b21, b12*b22, b13*b23]])

    M4 = np.array([[b22*b33+b23*b32, b21*b33+b23*b31, b22*b31+b21*b32],
                   [b12*b33+b13*b32, b11*b33+b13*b31, b11*b32+b12*b31],
                   [b22*b13+b12*b23, b11*b23+b13*b21, b22*b11+b12*b21]])

    M = np.block([[M1, 2*M2],
                  [M3, M4]])

    # Calculate the new stiffness tensor
    cnew = M @ c @ M.T

    return cnew, M


---

## Test x3 axis rotation (vertical rotation)

In [4]:
Cij_rot1, M1 = rotate_Cij(Cij_example, angle_degrees=30, rotation_axis=3)
np.around(Cij_rot1, 2)

array([[106.4 ,  52.97,  24.16,  -1.18,  -0.93,  11.21],
       [ 52.97, 165.47,  29.64,   2.87,  -4.21,  25.54],
       [ 24.16,  29.64, 171.9 ,  -1.69,   7.34, -14.86],
       [ -1.18,   2.87,  -1.69,  26.93,  -4.66,  -6.21],
       [ -0.93,  -4.21,   7.34,  -4.66,  29.77,  -5.22],
       [ 11.21,  25.54, -14.86,  -6.21,  -5.22,  42.37]])

In [5]:
Cij_rot2, M2 = rotate_Cij2(Cij_example, angle_degrees=30, rotation_axis=3)
np.around(Cij_rot2, 2)

array([[106.4 ,  52.97,  24.16,  -1.18,  -0.93,  11.21],
       [ 52.97, 165.47,  29.64,   2.87,  -4.21,  25.54],
       [ 24.16,  29.64, 171.9 ,  -1.69,   7.34, -14.86],
       [ -1.18,   2.87,  -1.69,  26.93,  -4.66,  -6.21],
       [ -0.93,  -4.21,   7.34,  -4.66,  29.77,  -5.22],
       [ 11.21,  25.54, -14.86,  -6.21,  -5.22,  42.37]])

In [6]:
# Matlab reimplementation for x3
Cij_rot3, M3 = rotate_Cij_fromMatlab(Cij_example, theta=30)
np.around(Cij_rot3, 2)

array([[106.4 ,  52.97,  24.16,  -1.18,  -0.93,  11.21],
       [ 52.97, 165.47,  29.64,   2.87,  -4.21,  25.54],
       [ 24.16,  29.64, 171.9 ,  -1.69,   7.34, -14.86],
       [ -1.18,   2.87,  -1.69,  26.93,  -4.66,  -6.21],
       [ -0.93,  -4.21,   7.34,  -4.66,  29.77,  -5.22],
       [ 11.21,  25.54, -14.86,  -6.21,  -5.22,  42.37]])

In [7]:
# assert that the rotated Cij are similar
np.allclose(Cij_rot1, Cij_rot2), np.allclose(Cij_rot2, Cij_rot3)

(True, True)

In [8]:
# assert that when rotated -30 we obtain the original tensor
[np.allclose(Cij_example, rotate_Cij(Cij_rot1, angle_degrees=-30, rotation_axis=3)[0]),
 np.allclose(Cij_example, rotate_Cij(Cij_rot2, angle_degrees=-30, rotation_axis=3)[0]),
 np.allclose(Cij_example, rotate_Cij(Cij_rot3, angle_degrees=-30, rotation_axis=3)[0])]            

[True, True, True]

In [9]:
np.around(M1, 2)

array([[ 0.75,  0.25,  0.  ,  0.  ,  0.  ,  0.87],
       [ 0.25,  0.75,  0.  ,  0.  ,  0.  , -0.87],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.87, -0.5 ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.87,  0.  ],
       [-0.43,  0.43,  0.  ,  0.  ,  0.  ,  0.5 ]])

In [10]:
# assert that the Bond transformation matrices are the same
np.allclose(M1, M2), np.allclose(M2, M3)

(True, True)

### x1 axis rotation

In [11]:
Cij_rot1, M1 = rotate_Cij(Cij_example, angle_degrees=30, rotation_axis=1)
np.around(Cij_rot1, 2)

array([[ 96.2 ,  43.83,  40.67,   5.21,   2.78,  -3.53],
       [ 43.83, 136.6 ,  49.44,  -7.26,  -5.13, -29.09],
       [ 40.67,  49.44, 156.61,   2.06,   4.55,  13.22],
       [  5.21,  -7.26,   2.06,  26.93,  -4.66,  -1.64],
       [  2.78,  -5.13,   4.55,  -4.66,  29.77,   6.01],
       [ -3.53, -29.09,  13.22,  -1.64,   6.01,  69.54]])

In [12]:
Cij_rot2, M2 = rotate_Cij2(Cij_example, angle_degrees=30, rotation_axis=1)
np.around(Cij_rot2, 2)

array([[  80.68,    7.16,   53.57,  -22.51,    0.53,  -26.48],
       [   7.16,   73.48,   25.72,  -85.73,   -6.14,   -5.56],
       [  53.57,   25.72,  258.02, -113.72,    0.71,  -47.6 ],
       [ -22.51,  -85.73, -113.72,  169.32,    5.39,   12.81],
       [   0.53,   -6.14,    0.71,    5.39,   33.1 ,    1.31],
       [ -26.48,   -5.56,  -47.6 ,   12.81,    1.31,   51.02]])

In [13]:
# assert that when rotated -30 we obtain the original tensor
[np.allclose(Cij_example, rotate_Cij(Cij_rot1, angle_degrees=-30, rotation_axis=1)[0]),
 np.allclose(Cij_example, rotate_Cij(Cij_rot2, angle_degrees=-30, rotation_axis=1)[0])]

[True, False]

In [14]:
np.around(M1, 2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75,  0.25,  0.  ,  0.  ,  0.87],
       [ 0.  ,  0.25,  0.75,  0.  ,  0.  , -0.87],
       [ 0.  ,  0.  ,  0.  ,  0.87, -0.5 ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.87,  0.  ],
       [ 0.  , -0.43,  0.43,  0.  ,  0.  ,  0.5 ]])

In [15]:
np.around(M2, 2)

array([[ 0.87,  0.  ,  0.  ,  0.  ,  0.  ,  0.5 ],
       [ 0.  ,  0.5 , -0.43,  0.43,  0.  ,  0.  ],
       [ 0.  ,  0.87,  0.75,  0.25,  0.  ,  0.  ],
       [ 0.  , -0.87,  0.25,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [-0.5 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.87]])

### x2 axis rotation

In [16]:
Cij_rot1, M1 = rotate_Cij(Cij_example, angle_degrees=30, rotation_axis=2)
np.around(Cij_rot1, 2)

array([[106.94,  34.01,  52.65,   2.71,  13.61,  -3.34],
       [ 34.01, 189.4 ,  27.49,  -9.46, -15.84,  -2.39],
       [ 52.65,  27.49, 132.66,  -0.8 ,  22.67,  -9.54],
       [  2.71,  -9.46,  -0.8 ,  22.42,  -3.55,   2.75],
       [ 13.61, -15.84,  22.67,  -3.55,  47.35,  -1.84],
       [ -3.34,  -2.39,  -9.54,   2.75,  -1.84,  36.68]])

In [17]:
Cij_rot2, M2 = rotate_Cij2(Cij_example, angle_degrees=30, rotation_axis=2)
np.around(Cij_rot2, 2)

array([[ 82.68,  21.29,  13.38,   6.93,   5.28,   1.31],
       [ 21.29, 271.72,   1.88,   0.41,   2.04,  -7.94],
       [ 13.38,   1.88, 116.61,  -5.21,   8.07,  -5.39],
       [  6.93,   0.41,  -5.21,  23.6 ,   1.45,  -4.71],
       [  5.28,   2.04,   8.07,   1.45,  32.49,  -0.34],
       [  1.31,  -7.94,  -5.39,  -4.71,  -0.34,  36.11]])

In [18]:
# assert that when rotated -30 we obtain the original tensor
[np.allclose(Cij_example, rotate_Cij(Cij_rot1, angle_degrees=-30, rotation_axis=2)[0]),
 np.allclose(Cij_example, rotate_Cij(Cij_rot2, angle_degrees=-30, rotation_axis=2)[0])]

[True, False]